In [ ]:

OPEN_API_KEY=''
import nest_asyncio

nest_asyncio.apply()

#Load data


file_path = '/workspaces/codespaces-jupyter/files/metagpt.pdf'


#Define LLM and embedding model
import os
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter

documents = SimpleDirectoryReader(input_files=[file_path]).load_data()

sentence_splitter=SentenceSplitter(chunk_size=1024)
nodes=sentence_splitter.get_nodes_from_documents(documents)

#Define LLM and Embedding model
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding


Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002", api_key=OPEN_API_KEY)
# Dummy call to _prepare_chat_with_tools to bypass error


#Defiine Summary Index and Vector Index
from llama_index.core import SummaryIndex, VectorStoreIndex
summary_index = SummaryIndex(nodes=nodes)
vector_index = VectorStoreIndex(nodes=nodes)

#Define Query Engines and Set Metadata
summary_query_engine = summary_index.as_query_engine(response_mode='tree_summarize', use_async=True)
vector_query_engine = vector_index.as_query_engine()

from llama_index.core.tools import QueryEngineTool
summary_tool = QueryEngineTool.from_defaults(query_engine=summary_query_engine, description="Summarization questions about the document")
vector_tool = QueryEngineTool.from_defaults(query_engine=vector_query_engine, description="Retrieving specific content from the document")

#Define Router query engine
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine=RouterQueryEngine(selectors=LLMSingleSelector.from_defaults(),query_engine_tools=[summary_tool,vector_tool],verbose=True)

#Define Agent
response=query_engine.query("What is the document about?")
print(str(response))
print(len(response.source_nodes))

vector_response=query_engine.query("How do agents work?")

print(str(vector_response))
print(len(vector_response.source_nodes))








TypeError: Can't instantiate abstract class OpenAI without an implementation for abstract method '_prepare_chat_with_tools'